**<h1 align="center">Xây dựng mô hình dự đoán lương</h1>**

### **Giới thiệu thành viên**
|   **MSSV**   |       **Họ Tên**      |
|:------------:|:---------------------:|
| **21280099** | Nguyễn Công Hoài Nam  |
| **21280118** | Lê Nguyễn Hoàng Uyên  |
| **21280124** | Huỳnh Công Đức        |
| **21280125** | Trần Thị Uyên Nhi     |

## **Chuẩn bị**

Import thư viện 

In [15]:
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import RobustScaler, StandardScaler, MaxAbsScaler, MinMaxScaler, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.linear_model import LinearRegression 

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.model_selection import cross_validate

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

Load dữ liệu 

In [16]:
df = pd.read_csv('../dataset/clean_data.csv')
df = df.drop(["Unnamed: 0"],axis=1)
print(f"Dataframe shape: {df.shape}")
df

Dataframe shape: (39772, 46)


,RemoteWork,EdLevel,YearsCodePro,DevType,Country,Age,Salary,HTML/CSS,JavaScript,Python,...,Yarn,Homebrew,Other toolsTech,Vim,Visual Studio Code,IntelliJ IDEA,Android Studio,Notepad++,Visual Studio,Other collabTool
0,Hybrid,Bachelor’s degree,7.0,"Developer, front-end",USA,25-34,156000.0,1.0,1.0,0.0,...,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
1,Remote,Bachelor’s degree,4.0,"Developer, full-stack",Other,25-34,23456.0,1.0,1.0,0.0,...,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2,Remote,Less than a Bachelors,21.0,"Developer, back-end",UK,35-44,96828.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,Remote,Less than a Bachelors,3.0,"Developer, full-stack",USA,35-44,135000.0,1.0,1.0,0.0,...,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
4,Remote,Bachelor’s degree,3.0,"Developer, full-stack",USA,25-34,80000.0,1.0,1.0,1.0,...,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39767,Hybrid,Bachelor’s degree,8.0,"Developer, front-end",Sweden,25-34,52981.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
39768,Hybrid,Post grad,5.0,"Developer, mobile",Other,25-34,28625.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0
39769,Remote,Master’s degree,24.0,"Developer, back-end",Brazil,35-44,50719.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
39770,Hybrid,Master’s degree,9.0,Other,France,25-34,64254.0,1.0,1.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0


## **Chia tập dữ liệu thành tập train và tập test**

In [17]:
# Lấy ngẫu nhiên khoảng 95% mẫu dữ liệu cho tập train
train = df.sample(frac=0.95, random_state=42)
#Phần còn lại cho tập test
test = df.drop(train.index)

print(f"Train shape: {train.shape}")
print(f"Test shape: {test.shape}")

print("_" * 25)
print()

# Tách cột lương ra ngoài để đưa vào mô hình 
X_train = train.drop(["Salary"], axis=1)
y_train = train["Salary"].values
X_test= test.drop(["Salary"], axis=1)
y_test = test["Salary"].values

print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")


Train shape: (37783, 46)
Test shape: (1989, 46)
_________________________

X_train shape: (37783, 45)
y_train shape: (37783,)
X_test shape: (1989, 45)
y_test shape: (1989,)


# **Chọn Model**

Hàm tính các metrics như `r2,mae,..` để đánh giá hiệu suất model

In [18]:
def print_metrics(metrics_dict):
    fit_times = metrics_dict['fit_time']
    score_times = metrics_dict['score_time']
    rmse = metrics_dict['test_neg_root_mean_squared_error']
    mae = metrics_dict['test_neg_mean_absolute_error']
    r2 = metrics_dict['test_r2']

    data = {
        'RMSE': [-1 * val for val in rmse],
        'MAE': [-1 * val for val in mae],
        'R-squared': r2
    }

    df = pd.DataFrame(data)

    mean_values = {
        'RMSE': -1 * np.mean(rmse),
        'MAE': -1 * np.mean(mae),
        'R-squared': np.mean(r2)
    }

    mean_df = pd.DataFrame(mean_values, index=['Mean'])
    df = pd.concat([df, mean_df])

    # Làm tròn các giá trị trong DataFrame
    df = df.round(decimals=4)

    return df

scoring = ["neg_root_mean_squared_error", "neg_mean_absolute_error", "r2"]

## **Quy trình**

Xây dựng một pineline thực hiện tuần tự gồm:
- **Transform**: Mã hóa và chuẩn hóa dữ liệu
- **Modeling**: Mô hình hóa dựa vào tập dữ liệu đã transform

Chi tiết hơn cho bước **Transform**:
- Mã hóa `OrdinalEncoder` tức mã hóa có thự tự để mã hóa các các cột có phân biệt thự tự 
- Mã hóa `OneHotEncoder` cho các cột không phân biệt thứ tự
- Chuẩn hóa `MinMaxScaler` cho cột `YearCodePro`



Đánh giá hiệu suất của quy trình đó sử dụng `K Fold Cross Validation`.

**Cross validation là một phương pháp để ước lượng hiệu suất của mô hình, từ đó tìm ra mô hình tốt nhất cho bài toán trong đó
KFold là một kỹ thuật trong CV cụ thể:**

1. Xáo trộn dataset một cách ngẫu nhiên
2. Chia dataset thành k nhóm (số k-fold)
3. Với mỗi nhóm:
    - Sử dụng nhóm hiện tại để đánh giá hiệu quả mô hình
    - Các nhóm còn lại được sử dụng để huấn luyện mô hình
    - Huấn luyện mô hình
    - Đánh giá và sau đó hủy mô hình
4. Tổng hợp hiệu quả của mô hình dựa từ các số liệu đánh giá

## **Linear Regression**

In [19]:
transform = ColumnTransformer([
    ("label", OrdinalEncoder(), ["EdLevel", "Country", "Age"]),
    ("onehot", OneHotEncoder(sparse_output=False, handle_unknown="ignore"), ["RemoteWork", "DevType"]),
    ("scaler", MaxAbsScaler(), ["YearsCodePro"])
], remainder="passthrough")

model = LinearRegression()

pipe = Pipeline([
    ("preprocess", transform),
    ("model", model)
])

kfold = KFold(n_splits=5, shuffle=True, random_state=42)

scores_lr = cross_validate(pipe, X_train, y_train, cv=kfold, scoring=scoring, n_jobs=2)
print_metrics(scores_lr)


,RMSE,MAE,R-squared
0,41023.1685,32765.7982,0.3018
1,41118.4926,32535.5486,0.2848
2,41419.4230,32800.0431,0.2750
3,40566.4660,32079.8033,0.2978
4,41093.7046,32449.7329,0.2699
Mean,41044.2510,32526.1852,0.2858


## **Decision Tree**

In [20]:
transform = ColumnTransformer([
    ("label", OrdinalEncoder(), ["EdLevel", "Country", "Age"]),
    ("onehot", OneHotEncoder(sparse_output=False, handle_unknown="ignore"), ["RemoteWork", "DevType"]),
    ("scaler", MaxAbsScaler(), ["YearsCodePro"])
], remainder="passthrough")

model = DecisionTreeRegressor(random_state=42)

pipe = Pipeline([
    ("preprocess", transform),
    ("model", model)
])

kfold = KFold(n_splits=5, shuffle=True, random_state=42)

scores_dt = cross_validate(pipe, X_train, y_train, cv=kfold, scoring=scoring, n_jobs=2)
print_metrics(scores_dt)


,RMSE,MAE,R-squared
0,46648.6914,34160.2641,0.0971
1,46194.6447,33835.3127,0.0973
2,46092.3383,33776.7477,0.1022
3,47112.0565,34480.2808,0.0529
4,46341.5014,33980.9485,0.0715
Mean,46477.8465,34046.7108,0.0842


Ở hai mô hình `Linear Regression` và `Decision Tree` cho hiệu số không khả quan nên thực hiện cải thiện bằng `Ensemble Learning`

**Ensemble Learning** là kỹ thuật kết hợp các mô hình lại với nhạu để cho ra mô hình cuối cùng, gồm
- **Bagging**: Xây dựng một lượng lớn các model (thường là cùng loại) trên những subsamples khác nhau từ tập training dataset (random sample trong 1 dataset để tạo 1 dataset mới). Những model này sẽ được train độc lập và song song với nhau nhưng đầu ra của chúng sẽ được trung bình cộng để cho ra kết quả cuối cùng. (`Bagging Regressor`, `Random Forest`)

- **Boosting**: Xây dựng một lượng lớn các model (thường là cùng loại). Mỗi model sau sẽ học cách sửa những errors của model trước (dữ liệu mà model trước dự đoán sai) -> tạo thành một chuỗi các model mà model sau sẽ tốt hơn model trước bởi trọng số được update qua mỗi model (`Gradient Boosting`. `Ada Boosting`)

- **Stacking**: Xây dựng một số model (thường là khác loại) và một meta model (supervisor model), train những model này độc lập, sau đó meta model sẽ học cách kết hợp kết quả dự báo của một số mô hình một cách tốt nhất.

## **AdaBoost**

In [21]:
transform = ColumnTransformer([
    ("label", OrdinalEncoder(), ["EdLevel", "Country", "Age"]),
    ("onehot", OneHotEncoder(sparse_output=False, handle_unknown="ignore"), ["RemoteWork", "DevType"]),
    ("scaler", MaxAbsScaler(), ["YearsCodePro"])
], remainder="passthrough")

model = AdaBoostRegressor(estimator=DecisionTreeRegressor(), n_estimators=200, random_state=42)

pipe = Pipeline([
    ("preprocess", transform),
    ("model", model)
])

kfold = KFold(n_splits=5, shuffle=True, random_state=42)

scores_ab = cross_validate(pipe, X_train, y_train, cv=kfold, scoring=scoring, n_jobs=2)
print_metrics(scores_ab)

/home/hnam/.local/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


,RMSE,MAE,R-squared
0,32516.7187,23536.2368,0.5613
1,32818.1386,23777.6293,0.5444
2,32740.6211,23743.8900,0.5470
3,32226.1906,23222.5057,0.5568
4,32833.6510,23835.8204,0.5339
Mean,32627.0640,23623.2164,0.5487


## **Bagging**

In [22]:
transform = ColumnTransformer([
    ("label", OrdinalEncoder(), ["EdLevel", "Country", "Age"]),
    ("onehot", OneHotEncoder(sparse_output=False, handle_unknown="ignore"), ["RemoteWork", "DevType"]),
    ("scaler", MaxAbsScaler(), ["YearsCodePro"])
], remainder="passthrough")
model = BaggingRegressor(estimator=DecisionTreeRegressor(), n_estimators=200, n_jobs=2, random_state=42)

pipe = Pipeline([
    ("preprocess", transform),
    ("model", model)
])

kfold = KFold(n_splits=5, shuffle=True, random_state=42)

scores_bg = cross_validate(pipe, X_train, y_train, cv=kfold, scoring=scoring, n_jobs=2)
print_metrics(scores_bg)

/home/hnam/.local/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


,RMSE,MAE,R-squared
0,32326.6405,24067.8948,0.5664
1,32762.7481,24334.2254,0.5459
2,32673.0239,24264.9800,0.5488
3,32226.5656,23756.7468,0.5568
4,32656.4067,24308.0575,0.5389
Mean,32529.0770,24146.3809,0.5514


## **RandomForest**

In [23]:
transform = ColumnTransformer([
    ("label", OrdinalEncoder(), ["EdLevel", "Country", "Age"]),
    ("onehot", OneHotEncoder(sparse_output=False, handle_unknown="ignore"), ["RemoteWork", "DevType"]),
    ("scaler", MaxAbsScaler(), ["YearsCodePro"])
], remainder="passthrough")

model = RandomForestRegressor(n_estimators=200, n_jobs=2, random_state=42)

pipe = Pipeline([
    ("preprocess", transform),
    ("model", model)
])

kfold = KFold(n_splits=5, shuffle=True, random_state=42)

scores_rf = cross_validate(pipe, X_train, y_train, cv=kfold, scoring=scoring, n_jobs=2)
print_metrics(scores_rf)

,RMSE,MAE,R-squared
0,32328.6558,24069.6346,0.5664
1,32783.2697,24345.6567,0.5453
2,32657.7412,24267.7525,0.5493
3,32251.0956,23783.5318,0.5561
4,32651.8420,24294.5406,0.5391
Mean,32534.5209,24152.2232,0.5512


## **Gradient Boost**

In [24]:
transform = ColumnTransformer([
    ("label", OrdinalEncoder(), ["EdLevel", "Country", "Age"]),
    ("onehot", OneHotEncoder(sparse_output=False, handle_unknown="ignore"), ["RemoteWork", "DevType"]),
    ("scaler", MaxAbsScaler(), ["YearsCodePro"])
], remainder="passthrough")

model = GradientBoostingRegressor(n_estimators=200)

pipe = Pipeline([
    ("preprocess", transform),
    ("model", model)
])

kfold = KFold(n_splits=5, shuffle=True, random_state=42)

scores_gb = cross_validate(pipe, X_train, y_train, cv=kfold, scoring=scoring, n_jobs=2)
print_metrics(scores_gb)

,RMSE,MAE,R-squared
0,32231.3481,24116.3954,0.5690
1,32150.7185,23911.1742,0.5627
2,32461.9212,24068.5509,0.5547
3,31727.4239,23342.2847,0.5704
4,32129.7571,23755.8375,0.5537
Mean,32140.2337,23838.8485,0.5621


## **Model tốt nhất**

In [25]:
def summary(scores_lists):
    metrics = {'RMSE': [], 'MAE': [], 'R2-SCORE': []}

    for scores in scores_lists:
        metrics['RMSE'].append(-1 * np.mean(scores['test_neg_root_mean_squared_error']))
        metrics['MAE'].append(-1 * np.mean(scores['test_neg_mean_absolute_error']))
        metrics['R2-SCORE'].append(np.mean(scores['test_r2']))
    
    metrics = pd.DataFrame(metrics, index=['Linear Regression', 'Decision Tree', 'Ada Boosting', 'Bagging', 'Random Forest', 'Gradient Boosting'])

    sorted_metrics = metrics.sort_values(by=['RMSE', 'MAE', 'R2-SCORE'], ascending=[False, False, True])

    return sorted_metrics


In [26]:
scores_lists = [scores_lr, scores_dt, scores_ab, scores_bg, scores_rf, scores_gb]
summary(scores_lists)

,RMSE,MAE,R2-SCORE
Decision Tree,46477.846461,34046.710778,0.084191
Linear Regression,41044.250955,32526.185233,0.285835
Ada Boosting,32627.064008,23623.216436,0.548684
Random Forest,32534.520855,24152.223230,0.551240
Bagging,32529.076982,24146.380884,0.551389
Gradient Boosting,32140.233749,23838.848545,0.562098


Từ số liệu ta có thể thấy `Gradient Boosting` cho hiệu suất tốt nhất

# **Hyperparameter Tuning**

Cải thiện thêm mô hình sử dụng `GridSearchCV`
- Duyệt qua tất cả các kết hợp có thể của các giá trị siêu tham số được chỉ định trước.
- Tìm kiếm siêu tham số tốt nhất dựa trên đánh giá của mô hình trên cross-validation.

In [27]:
transform = ColumnTransformer([
    ("label", OrdinalEncoder(), ["EdLevel", "Country", "Age"]),
    ("onehot", OneHotEncoder(sparse_output=False, handle_unknown="ignore"), ["RemoteWork","DevType"]),
    ("scaler", MaxAbsScaler(), ["YearsCodePro"])
], remainder="passthrough")

model = GradientBoostingRegressor(random_state=42)

params = {
    "n_estimators": [*range(200, 510, 50)],
    "loss": ['squared_error','absolute_error', 'huber', 'quantile'],
    "learning_rate": [0.01, 0.1, 0.2, 0.3, 0.4],
    "criterion": ['friedman_mse', 'squared_error']

}

grid = GridSearchCV(estimator=model, param_grid=params, scoring=scoring, n_jobs=-1, verbose=1, cv=3, refit="r2", error_score='raise')

pipe = Pipeline([
    ("preprocess", transform),
    ("grid", grid)
])

pipe.fit(X_train.head(10000), y_train[:10000])
print(f"The best params: {pipe['grid'].best_params_}")
print(f"The best score: {pipe['grid'].best_score_}")

Hyperparameter mà thuật toán tìm được: 
- `criterion` = `friedman_mse`'
- `learning_rate` = `0.1`
- `loss` = `huber`
- `n_estimators` = `500`

# **Train & Save Best Model**

In [28]:
from sklearn import metrics 

def evaluate(y_true, y_pred):
    rmse = metrics.mean_squared_error(y_true=y_true, y_pred= y_pred, squared=False)
    mae = metrics.mean_absolute_error(y_true, y_pred)
    r2 = metrics.r2_score(y_true, y_pred)

    metrics_dict = {
        "Metrics": ["Root Mean Square Error (RMSE)", 
                    "Mean Absolute Error (MAE)", 
                    "R2-score (R2)"],
        "Values": [rmse, 
                    mae, 
                    r2]
    }

    metrics_df = pd.DataFrame(metrics_dict)
    print(metrics_df)

In [29]:
transform = ColumnTransformer([
    ("label", OrdinalEncoder(), ["EdLevel", "Country", "Age"]),
    ("onehot", OneHotEncoder(sparse_output=False, handle_unknown="ignore"), ["RemoteWork", "DevType"]),
    ("scaler", MaxAbsScaler(), ["YearsCodePro"])
], remainder="passthrough")

model = GradientBoostingRegressor(criterion='friedman_mse', 
                                learning_rate=0.1, 
                                loss='huber', 
                                n_estimators= 400)

pipe = Pipeline([
    ("preprocess", transform),
    ("model", model)
])

pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)
evaluate(y_test, y_pred)

                         Metrics        Values
0  Root Mean Square Error (RMSE)  31743.204291
1      Mean Absolute Error (MAE)  22890.788968
2                  R2-score (R2)      0.572137


Nhìn chung, hiệu suất của model vẫn chưa đạt ngưỡng kỳ vọng

## **Lưu lại model để thực hiện dự đoán**


In [30]:
joblib.dump(pipe, "../dataset/best_model.joblib")

['../dataset/best_model.joblib']

## **Website dự đoán**
<h1 align="center">
  <strong>Quét mã QR dưới đây hoặc nhập link</strong>
  <br>
</h1>

<p align="center">
  <br>  <img src="../app/static/qr.svg" alt="Logo OpenAI" width="600" height="600">
  <br>
</p>
